In [1]:
import openai
from utils import GmapsUtils
from dotenv import dotenv_values

In [2]:

env_vars = dotenv_values(".env")
OPENAI_API_KEY = env_vars["OPENAI_API_KEY"]
openai.api_key = OPENAI_API_KEY

In [13]:
# start = "Karlsruhe HBF"
# end = "Tokyo"

# (t_dp, t_gm, t_di, t_du) = GmapsUtils.calculate_route_gmaps(
#     start,
#     end,
#     'transit'
# )

# print(f"{t_dp=}")


GMAPS FAILED
t_dp=None


In [3]:


def get_environment_score():
    # start = request.args.get("start")
    # end = request.args.get("end")
    from_loc = "Karlsruhe HBF"
    to_loc = "Istanbul"

    # from_loc = request.args.get("from") # get data from frontend
    # to_loc = request.args.get("to") # get data from frontend


    (w_dist, w_dur, w_wp, w_r) = GmapsUtils.calculate_route_gmaps(
        from_loc,
        to_loc,
        'walking'
    )
    print(f"{w_dist=}")
    (b_dist, b_dur, b_wp, b_r) = GmapsUtils.calculate_route_gmaps(
        from_loc,
        to_loc,
        'biking'
    )
    print(f"{b_dist=}")

    (d_dist, d_dur, d_wp, d_r) = GmapsUtils.calculate_route_gmaps(
        from_loc,
        to_loc,
        'driving'
    )
    print(f"{d_dist=}")

    (p_dist, p_dur, p_wp, p_r) = GmapsUtils.calculate_route_gmaps(
        from_loc,
        to_loc,
        'transit'
    )
    print(f"{p_dist=}")



    text = f"You have to travel from {from_loc} to {to_loc}. Please rate each method with a singular score of 0 (least likely) to 100 (most likely) which transportation you would like to take if you consider CO2 emissions and travel time. You are an environmentally friendly person, but if the travel time is long or unrealistic, you prefer faster options. The following travel methods are available: walking, bicycle, driving, public_transportation, plane Give the results only (one score per travel method) back in JSON format."
    
    if w_dist is not None:
        text += f"\nTake the following information into consideration:\n"
        text += f"(walking distance (m), walking duration (min))={(w_dist, w_dur)}"
        text += f"(public transportation distance (m), public transportation duration (min))={(p_dist, p_dur)}"
        text += f"(driving distance (m), driving duration (min))={(d_dist, d_dur)}"
        text += f"(bicycle distance (m), bicycle duration (min))={(b_dist, b_dur)}"



    message = [{"role": "system", "content": text}]

    print(f"{message=}")
    chat = openai.ChatCompletion.create(
        # model="gpt-4",
        model="gpt-3.5-turbo",
        messages=message
    )
    reply = chat.choices[0].message.content

    print(f"{reply=}")
    message.append({"role":"assistant", "content": reply})

    

    text_2 = f"Now provide a catastrophe score for how bad the climate change effects would be if the entire humanity took comparable routes every day. Give a score between 0 and 100 for each method. Give the results only (one score per travel method) in JSON format"

    text_2 += f"For the method 'car', also take into consideration the amount of maneuvers (left and right turns) it would take to realize the route whilst calculating the score. Amount of manuevers for method car: {len(GmapsUtils.get_maneuvers(d_r))}"
    

    message.append({"role":"system", "content": text_2})

    chat_2 = openai.ChatCompletion.create(
        # model="gpt-4",
        model="gpt-3.5-turbo",

        messages=message
    )
    reply_2 = chat_2.choices[0].message.content


    response_object = {}
    response_object['reply_1'] = reply
    response_object['reply_2'] = reply_2 
    return response_object


In [8]:
score_responses

NameError: name 'score_responses' is not defined

In [4]:
import json
score_responses = get_environment_score()



efficiency_scores = json.loads(score_responses.get('reply_1'))
catastrophy_scores = json.loads(score_responses.get('reply_2'))

w_dist=2189463
b_dist=2186150
d_dist=2186150
p_dist=2288055
message=[{'role': 'system', 'content': 'You have to travel from Karlsruhe HBF to Istanbul. Please rate each method with a singular score of 0 (least likely) to 100 (most likely) which transportation you would like to take if you consider CO2 emissions and travel time. You are an environmentally friendly person, but if the travel time is long or unrealistic, you prefer faster options. The following travel methods are available: walking, bicycle, driving, public_transportation, plane Give the results only (one score per travel method) back in JSON format.\nTake the following information into consideration:\n(walking distance (m), walking duration (min))=(2189463, 1609495)(public transportation distance (m), public transportation duration (min))=(2288055, 140133)(driving distance (m), driving duration (min))=(2186150, 80883)(bicycle distance (m), bicycle duration (min))=(2186150, 80883)'}]
reply='{\n  "walking": 0,\n  "bicycle": 

In [9]:
efficiency_scores

{'walking': 0,
 'bicycle': 10,
 'driving': 40,
 'public_transportation': 80,
 'plane': 100}

In [8]:
catastrophy_scores['walking']

0

In [6]:
catastrophy_scores

{'walking': 0,
 'bicycle': 5,
 'driving': 90,
 'public_transportation': 30,
 'plane': 95}

In [ ]:
# karlsruhe - berlin (16 seconds)

In [ ]:
# ('{\n  "walking": 0,\n  "bicycle": 50,\n  "plane": 10,\n  "driving": 60,\n  "public transportation": 100\n}',
#  '{\n  "walking": 0,\n  "bicycle": 5,\n  "plane": 100,\n  "driving": 90,\n  "public transportation": 30\n}')

In [ ]:
# ('{\n  "walking": 0,\n  "bicycle": 10,\n  "plane": 70,\n  "driving": 35,\n  "public_transportation": 100\n}',
#  '{\n  "walking": 0,\n  "bicycle": 5,\n  "plane": 100,\n  "driving": 80,\n  "public_transportation": 30\n}')

In [ ]:
# ('{\n  "walking": 0,\n  "bicycle": 25,\n  "plane": 95,\n  "driving": 7,\n  "public transportation": 90\n}',
#  '{\n  "walking": 0,\n  "bicycle": 10,\n  "plane": 96,\n  "driving": 81,\n  "public transportation": 18\n}')

In [ ]:
# karlsruhe to nanjing china